In [1]:
import bz2 
import pickle
import _pickle as cPickle
import os
import pandas as pd
from itertools import chain
import numpy as np

#modified code for desktop usage
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chenx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
### Read in Files

path = os.getcwd()
folder = '\\Data'
files = os.listdir(path + folder)

files

['OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C105795698.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C111368507.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C12554922.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C144024400.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C153294291.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C184779094.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C8058405.pbz2',
 'OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_C97355855.pbz2']

In [3]:
# load in cPickle file for Geophysics (OpenAlex ID C8058405)
#discipline = 'C8058405'



# load in cPickle file for Statistics (OpenAlex ID C105795698)
#discipline = 'C105795698'

# load in cPickle file for Oceanography (OpenAlex ID C111368507)
#discipline = 'C111368507'

# load in cPickle file for Biophysics (OpenAlex ID C12554922)
discipline = 'C12554922'

# load in cPickle file for Meteorology (OpenAlex ID C153294291)
#discipline = 'C153294291'

# load in cPickle file for Atomic physics (OpenAlex ID C184779094)
#discipline = 'C184779094'

# load in cPickle file for Thermodynamics (OpenAlex ID C97355855)
#discipline = 'C97355855'

Data_Packet = 'Data\OUTPUT_Python_OpenAlex_Citation_and_Abstract_Data_' + discipline + '.pbz2'

f = bz2.BZ2File(Data_Packet, 'rb')
paper_id_year_df = cPickle.load(f)
corpus_dict = cPickle.load(f)
citation_df = cPickle.load(f)

# Object 1: dataframe with all the paper IDs and the year they were published: needed for the corpus_dict
geophysics_paper_id_year_df = paper_id_year_df
display(geophysics_paper_id_year_df.head())

# Object 2: dictionary where the keys are the paper IDs and the values are a list containing the extracted terms
# Structure: corpus_dict[Discipline_ID][paper_id] = [term1, term2, term3,...]: incl. eN and non-EN terms
geophysics_corpus_dict = corpus_dict

# sample call for single work
#display(geophysics_corpus_dict.get('C8058405').get('https://openalex.org/W2765252368'))

# full call for all works
#display(next(iter(geophysics_corpus_dict.items())))

# Object 3: dataframe that's an edgelist between receiver RORs and sender RORs per year (= research organization registry)
display(citation_df.head())

,work_id,publication_year,Discipline
0,https://openalex.org/W2775293320,2017,C12554922
1,https://openalex.org/W2625325858,2017,C12554922
2,https://openalex.org/W2478767193,2016,C12554922
3,https://openalex.org/W2109735518,2012,C12554922
4,https://openalex.org/W2037483496,2012,C12554922


,Sender_ROR,Receiver_ROR,Year,Citations,Discipline
0,https://ror.org/05apxxy63,https://ror.org/01y2jtd41,1940,1.0,C12554922
1,https://ror.org/00b30xv10,https://ror.org/00b30xv10,1941,4.5,C12554922
2,https://ror.org/00hj8s172,https://ror.org/00b30xv10,1941,0.5,C12554922
3,https://ror.org/00hx57361,https://ror.org/00b30xv10,1941,3.0,C12554922
4,https://ror.org/00py81415,https://ror.org/00b30xv10,1941,2.0,C12554922


In [4]:
### Preprocess Data for single field

def reformat(dictionary):
    '''
    Takes the Object 2 dictionary and casts it into a dataframe
    '''
    # remove OpenAlex ID
    field_dictionary = dictionary[1]
    # reformat to list
    field_list = list(map(list, field_dictionary.items()))
    # cast into dataframe and rename columns
    field_df = pd.DataFrame(field_list)
    field_df = field_df.rename(columns = {0: 'work_id', 1: 'terms'})
    
    return field_df

def add_year(field_df, paper_id_year_df):
    '''
    Takes the Object 2 dataframe and merges it with corresponding publication years from Object 1
    '''
    # merge the two dataframes
    merged = pd.merge(field_df, paper_id_year_df, on ='work_id', how ='inner')
    # reorder columns
    columns = merged.columns.tolist()
    columns = columns[-1:] + columns[-2:-1] + columns[:-2]
    reordered = merged[columns]
    
    return reordered

def lowercase(dataframe, column):
    '''
    takes a dataframe and lowercases everything within a specified column (column contents must be in a list of strings)
    '''
    dataframe[column] = dataframe[column].apply(lambda lst: [word.lower() for word in lst])
    return dataframe

def counts_per_document(reordered):
    '''
    Add number of documents, terms, unique terms, words, and unique words per document to the dataframe
    '''
    pd.set_option('mode.chained_assignment',None)
    
    reordered.loc[:,'NoD_pD'] = 1
    reordered.loc[:,'NoT_pD'] = [len(cell) for cell in reordered['terms']]
    reordered.loc[:,'NoUT_pD'] = [len(set(cell)) for cell in reordered['terms']]
    reordered.loc[:,'NoW_pD'] = [sum([len(term.split()) for term in cell]) for cell in reordered['terms']]
    reordered.loc[:,'NoUW_pD'] = [len(set([item for sublist in [term.split() for term in cell]
                                           for item in sublist])) for cell in reordered['terms']]
    
    #display(reordered.describe())
    return reordered

def counts_per_year(reordered):
    '''
    Add number of documents, terms, unique terms, words, and unique words per year to the dataframe
    '''
    # aggregate documents per year and concatenate the list(s) of words
    words = reordered.groupby('publication_year', as_index=False)['stemmed_tokens'].agg(lambda x: list(chain.from_iterable(x)))
    # aggregate documents per year and count the number of documents
    documents = reordered.groupby('publication_year', as_index=False).size()
    # put the two dataframes together
    grouped = pd.concat([words, documents['size']], axis = 1)
    
    # get counts of terms and words per year
    grouped = grouped.rename(columns = {'size':'NoD'})
    #grouped.loc[:,'NoT'] = [len(cell) for cell in grouped['terms']]
    #grouped.loc[:,'NoUT'] = [len(set(cell)) for cell in grouped['terms']]
    #grouped.loc[:,'NoW'] = [sum([len(term.split()) for term in cell]) for cell in grouped['words']]
    #grouped.loc[:,'NoUW'] = [len(set([item for sublist in [term.split() for term in cell]
                                      #for item in sublist])) for cell in grouped['words']]
    grouped.loc[:,'NoS'] = [sum([len(term.split()) for term in cell]) for cell in grouped['stemmed_tokens']]
    grouped.loc[:,'NoUS'] = [len(set([item for sublist in [term.split() for term in cell]
                                      for item in sublist])) for cell in grouped['stemmed_tokens']]
    
    return grouped

def split_string(dataframe, column):
    '''
    Split strings into substrings for a given column in the dataframe, creating the new column 'words'
    '''
    dataframe['words'] = dataframe[column].apply(lambda lst: [word for line in lst for word in line.split()])
    return dataframe

def remove_stopwords(dataframe, column):
    '''
    Remove stopwords from a list of words
    '''
    dataframe[column] = dataframe[column].apply(lambda lst: [word for word in lst if word not in stopwords])
    return dataframe

def wordcounter(wordlist, n):
    '''
    Counts terms/words within a list of strings, returns top n terms/words over time
    Idea: Use output as illustrative example of how field progresses (validate with field-specific paper on paradigm shift)
    '''
    counts = {}
    for word in wordlist:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    
    # convert dictionary to list of tuples
    lst_counts = [(key, value) for key, value in counts.items()]
    #sort in descending order
    lst_counts.sort(key = lambda x: x[1], reverse=True)
    
    return lst_counts[:n]

def wordcounter_column(dataframe, column, n):
    '''
    Apply wordcounter() function to the entire column of a dataframe, returns a new column with top n items per year
    '''
    # define the new column name and fill it with nan values
    if n != 1:
        new_column = 'top ' + str(n) + ' ' + column
    else:
        new_column = 'top ' + str(n) + ' ' + column[:-1]
    dataframe[new_column] = np.nan
    
    # loop through each row to get most frequent words
    for index, row in dataframe.iterrows():
        dataframe.iloc[index,dataframe.columns.get_loc(new_column)] = [wordcounter(row[column], n)]    
    
    # above line throws an error if outer brackets are removed, the followinf code flattens the nested list
    # dataframe[new_column] =  dataframe[new_column].apply(np.ravel)
    
    return dataframe

def wordcounter_abs_and_perc(dataframe, column, n, percentage):
    '''
    UPDATED VERSION OF WORDCOUNTER_COLUMN
    
    Apply wordcounter() function to the entire column of a dataframe, returns a new column with either
    top n items per year or top n percent of items per year
    
    Function takes in a dataframe, a column name ('words' or 'terms'), n (either as whole number of cases or as percentage,
    and boolean percentage ('YES' or 'NO' to indicate if number is top n or top n percent))
    '''    
    # define the new column name conditional on percentage
    if percentage == 'YES':
        # get number of unique words/terms based on given percentage
        new_counter = 'NoU' + str(column[0]).capitalize() + ' (t' + str(n) + '%)'        
        new_column = 't' + str(n) + '% of ' + column
    elif n!= 1:
        new_column = 't' + str(n) + ' ' + column
    else:
        new_column = 't' + str(n) + ' ' + column[:-1]
        
    # populate new_counter column with an integer of terms, if percentage given
    if percentage == 'NO':
        pass
    #elif column == 'terms':
        #dataframe[new_counter] = dataframe['NoUT'].multiply((n/100)).round().astype(np.int64)
    #elif column == 'words':
        #dataframe[new_counter] = dataframe['NoUW'].multiply((n/100)).round().astype(np.int64)
    elif column == 'stemmed_tokens':
        dataframe[new_counter] = dataframe['NoUS'].multiply((n/100)).round().astype(np.int64)
        
    # fill other column with nan values
    dataframe[new_column] = np.nan
    
    # loop through each row to get most frequent words
    for index, row in dataframe.iterrows():        
        # condition for top n % of terms
        if percentage == 'YES':
            NoUX = dataframe.iloc[index,dataframe.columns.get_loc(new_counter)]
            # account for edge case of NoUT being 0
            if NoUX >= 1:
                dataframe.iloc[index,dataframe.columns.get_loc(new_column)] = [wordcounter(row[column], NoUX)]
            else:
                dataframe.iloc[index,dataframe.columns.get_loc(new_column)] = np.nan
        # condition for top n terms
        else:
            dataframe.iloc[index,dataframe.columns.get_loc(new_column)] = [wordcounter(row[column], n)]
            
        # above line throws an error if outer brackets are removed, the following code flattens the nested list
        # dataframe[new_column] =  dataframe[new_column].apply(np.ravel)
        
    return dataframe

In [5]:
#import nltk
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')

regex = r'[^a-z\s]'

def tokenize(text):
    '''
    Cleans, tokenizes + stems Pandas series of strings    
    Returns pandas series of lists of tokens
    '''
    # Clean text with regex
    clean = text.str.lower().str.replace(regex, '', regex=True)

    # Anonymous tokenizer + stemmer functions
    stop = nltk.corpus.stopwords.words('english')
    tokenize = lambda text: [i for i in nltk.word_tokenize(text) if i not in stop]
    stemmer = lambda tokens: [SnowballStemmer('english').stem(token) for token in tokens]

    # Tokenize and stem clean text
    tokens = clean.apply(tokenize)
    stemmed_tokens = tokens.apply(stemmer)
    
    return stemmed_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chenx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
a = lowercase(add_year(reformat(next(iter(geophysics_corpus_dict.items()))), geophysics_paper_id_year_df), 'terms')

# remove rows with empty lists
b = a[a['terms'].map(lambda d: len(d)) > 0]

# reformat row as strings
b = b.astype({'terms':'string'})

# split terms into words and stem
b['stemmed_tokens'] = tokenize(b['terms'])

# get ocunts per year for documents, words ,adn unique words (now no more terms because of stemming)
b = counts_per_year(b)

# top 10, 50, 100, 500 unique stemmed tokens
c = wordcounter_abs_and_perc(b, 'stemmed_tokens', 10, 'NO')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 50, 'NO')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 100, 'NO')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 500, 'NO')

# top 1%, 10%, 20%, 25 % of unique stemmed tokens
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 1, 'YES')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 10, 'YES')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 20, 'YES')
c = wordcounter_abs_and_perc(c, 'stemmed_tokens', 25, 'YES')

# flatten list, calculate len, and divide by 2 since list contains words and their count (should max at 100)
#c.loc[:,'t100 tokens count'] = c['t100 words'].apply(np.ravel).apply(len).div(2).astype(np.int64)
# select relevant keys and set publication_year to index
#d = c[['publication_year', 'NoD', 'NoT', 'NoUT', 'NoW', 'NoUW', 't100 words count', 't100 terms count', 'NoUW (t25%)',
      #'NoUT (t25%)']]

# fill NaN values with 0 (cases where terms do not meet threshold for meaningful results for a given top percentage)
c.fillna(0)

# convert publication year back to integer
c = c.astype({'publication_year':'int'})

d = c.set_index('publication_year')

# create complete index without missing years
new_index = list(range(int(min(d.index)), int(max(d.index)) + 1))

# create empty dataframe with complete index
e = pd.DataFrame(np.nan, index = new_index, columns = d.columns)

e.index.name = 'publication_year'

f = e.combine_first(d)
f.reset_index(inplace=True)

# display NA instead of full list of tokens (needed for usability as CSV for Felix)
f['stemmed_tokens'] = 'NA'

In [8]:
f

,publication_year,stemmed_tokens,NoD,NoS,NoUS,t10 stemmed_tokens,t50 stemmed_tokens,t100 stemmed_tokens,t500 stemmed_tokens,NoUS (t1%),t1% of stemmed_tokens,NoUS (t10%),t10% of stemmed_tokens,NoUS (t20%),t20% of stemmed_tokens,NoUS (t25%),t25% of stemmed_tokens
0,1842,NA,1.0,25.0,9.0,"[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...",0.0,NaN,1.0,"[(singl, 6)]",2.0,"[(singl, 6), (air, 5)]",2.0,"[(singl, 6), (air, 5)]"
1,1843,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1844,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1845,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1846,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2019,NA,13343.0,294355.0,9241.0,"[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...",92.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",924.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",1848.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",2310.0,"[[(cell, 7869), (protein, 3832), (membran, 334..."
178,2020,NA,16042.0,353675.0,9992.0,"[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...",100.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",999.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",1998.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",2498.0,"[[(cell, 9252), (protein, 4577), (membran, 404..."
179,2021,NA,16459.0,362228.0,10134.0,"[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...",101.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",1013.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",2027.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",2534.0,"[[(cell, 9165), (protein, 4849), (membran, 400..."
180,2022,NA,7218.0,159473.0,6572.0,"[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...",66.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",657.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",1314.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",1643.0,"[[(cell, 3813), (protein, 2282), (membran, 161..."


In [9]:
## Normally convert to csv

# f.to_csv('biophysics_df_304.csv')

In [10]:
'''
EXPLANATION

publication_year denotes the year of publication, starting with the earliest available data. For years after the first
observation for which data is not available, all entries are coded as NaN

stemmed_tokens is the full list of stemmed tokens, including tokens that were used multiple times

NoD counts then umber of documents
NoS counts the number of stemmed tokens
NoUS counts the number of unique stemmed tokens

t10 stemmed_tokens, t50 stemmed_tokens, t100 stemmed_tokens, and t500 stemmed_tokens represent the top 10, 50, 100, and 500
tokens by frequency, in the format of [('token', frequency count), ('token2', frequency count), etc.]. For years with a
number of unique stemmed tokens below the top n, the full number of tokens will be displayed
(e.g., 11 tokens for t50 stemmed_tokens in 1832)

NoUS (t1%), NoUS (t10%), NoUS (t20%), and NoUS (t25%) count the number of unique stemmed tokens in the top 1, 10, 20, and 25
percent. This can be 0 for smaller corpus sizes (e.g., top 1% for 11 unique stemmed tokens is 0, hence NoUS (t1%) is 0)

t1% of stemmed_tokens, t10% of stemmed_tokens, t20% of stemmed_tokens, and t25% of stemmed_tokens list the actual tokens
with their frequency of occurence, same as t10 stemmed_tokens (and subsequent columns) did before 
'''

"\nEXPLANATION\n\npublication_year denotes the year of publication, starting with the earliest available data. For years after the first\nobservation for which data is not available, all entries are coded as NaN\n\nstemmed_tokens is the full list of stemmed tokens, including tokens that were used multiple times\n\nNoD counts then umber of documents\nNoS counts the number of stemmed tokens\nNoUS counts the number of unique stemmed tokens\n\nt10 stemmed_tokens, t50 stemmed_tokens, t100 stemmed_tokens, and t500 stemmed_tokens represent the top 10, 50, 100, and 500\ntokens by frequency, in the format of [('token', frequency count), ('token2', frequency count), etc.]. For years with a\nnumber of unique stemmed tokens below the top n, the full number of tokens will be displayed\n(e.g., 11 tokens for t50 stemmed_tokens in 1832)\n\nNoUS (t1%), NoUS (t10%), NoUS (t20%), and NoUS (t25%) count the number of unique stemmed tokens in the top 1, 10, 20, and 25\npercent. This can be 0 for smaller cor

# Start Felix's code

In [11]:
## This is the processing unit copied from dataprocessing v3, MODIFIED for use here.

# NEW IMPORTATIONS
from openpyxl.utils.dataframe import dataframe_to_rows


# IMPORTATIONS
from openpyxl import Workbook, load_workbook
from openpyxl.utils.exceptions import InvalidFileException as FileExc


from decimal import *
from scipy.special import softmax
from scipy.stats import entropy
import numpy as np


In [12]:
# ___________________________________________________________
# -----------------------------------------------------------
#   NEW FILE + YEAR RANGE (START, YEARS) + START_ROW
# -----------------------------------------------------------

# FUNCTION: check that input is integer
def check_integer(inp):
    try:
        inp = int(inp)
        inp_is_int = True
    except ValueError:
        inp_is_int = False
    return inp_is_int

# New file with (1) Start year = START ; (2) No. of years/sheets = YEARS
wb = Workbook()

START = input("Input start year (inclusive): ")
while True:
    if check_integer(START):
        START = int(START)
        break
    else:
        START = input("Not integer. Input integer: ")

YEARS = 0
TERMIN = input("Input end year (inclusive): ")
while True:
    if check_integer(TERMIN):
        YEARS = int(TERMIN) - START + 1
        break
    else:
        TERMIN = input("Not integer. Input integer: ")

# New workbook xlsx file name = old one + "_processed.xlsx"
FILE_NAME = "biophysics_processed"

# Create a sheet for each year
for i in range(YEARS):
    ws_temp = wb.create_sheet(str(START+i))


Input start year (inclusive): 2000
Input end year (inclusive): 2020


In [13]:
# ___________________________________________________________
# -----------------------------------------------------------
#   LOAD ORIGINAL DATA FROM PANDAS DATAFRAME f
# -----------------------------------------------------------

# NEW: Start a separate Workbook for the original data (BEGIN templi + unpack)

orig_wb = Workbook()
orig_ws = orig_wb.active

datasheet = orig_ws

templi = list(dataframe_to_rows(f, index=False, header=True))

#print(list(dataframe_to_rows(f, index=False, header=True)))

In [14]:
# Convert templi to worksheet
'''
for i in range(len(templi)):
    for j in range(len(templi[0])):
        orig_ws.cell(row=i+1, column=j+1, value=str(templi[i][j]))
'''

'\nfor i in range(len(templi)):\n    for j in range(len(templi[0])):\n        orig_ws.cell(row=i+1, column=j+1, value=str(templi[i][j]))\n'

In [15]:
# datasheet['Q179'].value
print(
    type(templi[179][16]),
    str(templi[179][16][0])[:20]+'...'+str(templi[179][16][0])[-20:], '\n',
    
    type(templi[179][16][0][0]),
    templi[179][16][0][0],
    templi[179][16][0][-1],
)

<class 'list'> [('cell', 9252), ('p...4), ('infarct', 14)] 
 <class 'tuple'> ('cell', 9252) ('infarct', 14)


In [16]:
# Unpacking the double-nested lists
for i in range(len(templi)):
    for j in range(len(templi[0])):
        if type(templi[i][j]) == list and len(templi[i][j]) == 1:
            templi[i][j] = templi[i][j][0]
            

In [17]:
#datasheet['Q179'].value  (unpacked)
print(
    type(templi[179][16]),
    str(templi[179][16])[:20]+'...'+str(templi[179][16])[-20:], '\n',
    
    type(templi[179][16][0]),
    templi[179][16][0]
)

<class 'list'> [('cell', 9252), ('p...4), ('infarct', 14)] 
 <class 'tuple'> ('cell', 9252)


In [18]:
# MODIFIED CODE FOR NOTEBOOK SHEET (templi instead of datasheet)

# Identify start row
START_ROW = 1

def start_row_id(start_year, data_list):
    for i in range(1, 195):
        if data_list[i][0] == int(start_year):
            start_row = i
            break
    return i

START_ROW = start_row_id(START, templi)

START_ROW

159

In [19]:
# MODIFIED CODE FOR NOTEBOOK SHEET (templi instead of datasheet)

# Function to process a single cell (adapted for list-in-list format)
def process_cell_data(row, col, data_list, wksht):
    '''
    row and col are int, data_list is a list.
    Extract one cell from data_list > Deconstruct > Write in new file
    '''
    # Report on progress
    print(f'r{row}c{col}', end=' ')
    
    # Extract a list of tuples
    cell_list = data_list[row][col]
    
    # Iterate and extract from each tuple
    for i in range(len(cell_list)):
        # Add stem to one column (A)
        wksht['A'+str(i+1)] = cell_list[i][0]
        # Add absolute frequency to other column (B)
        wksht['B'+str(i+1)] = cell_list[i][1]
        # Add relative frequency (/NoD) to third column (C)
        wksht['C'+str(i+1)] = cell_list[i][1] / data_list[row][2]

In [20]:
# Back to Main code: for Notebook sheet, Q for top 25%
for i in range(YEARS):
    process_cell_data(row=i+START_ROW, col=16, data_list=templi, wksht=wb[str(START+i)])

r159c16 r160c16 r161c16 r162c16 r163c16 r164c16 r165c16 r166c16 r167c16 r168c16 r169c16 r170c16 r171c16 r172c16 r173c16 r174c16 r175c16 r176c16 r177c16 r178c16 r179c16 

In [21]:
# Define Function for rounding 4/5 (half up)
def rounding(number, precision):
    '''
    number float, precision float
    '''
    ret = float(
        Decimal(str(number)).quantize(
            Decimal(str(precision)), rounding=ROUND_HALF_UP
            )
        )
    return ret

In [22]:
wb.save(FILE_NAME+".xlsx")

In [24]:
# ___________________________________________________________
# -----------------------------------------------------------
#   Calculate Shannon ENTROPY (base e)
# -----------------------------------------------------------

# Create new list for the entropies
entropy_li = []

def entropy_list_append(year, entr_li, wkbook):
    col = list(wkbook[str(year)]['C'])
    col_vals = np.array([i.value for i in col])
    entr_rounded = rounding(entropy(col_vals), 0.001)
    entr_li.append(entr_rounded)

for d in range(YEARS):
    entropy_list_append(START+d, entropy_li, wb)

# Create new worksheet for entropy
entr_ws = wb.create_sheet("entropy", 0)

entr_ws['A2'] = "Entropy_0"

# Add the new entropies to the new worksheet
for d in range(YEARS):
    entr_ws.cell(row = 1, column = d+2, value = START+d)
    entr_ws.cell(row = 2, column = d+2, value = entropy_li[d])


wb.save(FILE_NAME+'.xlsx')
# ------------------- FILE IS SAVED HERE ---------------------


In [25]:
f

,publication_year,stemmed_tokens,NoD,NoS,NoUS,t10 stemmed_tokens,t50 stemmed_tokens,t100 stemmed_tokens,t500 stemmed_tokens,NoUS (t1%),t1% of stemmed_tokens,NoUS (t10%),t10% of stemmed_tokens,NoUS (t20%),t20% of stemmed_tokens,NoUS (t25%),t25% of stemmed_tokens
0,1842,NA,1.0,25.0,9.0,"[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...","[(singl, 6), (air, 5), (bronchial, 5), (ramif,...",0.0,NaN,1.0,"[(singl, 6)]",2.0,"[(singl, 6), (air, 5)]",2.0,"[(singl, 6), (air, 5)]"
1,1843,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1844,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1845,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1846,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2019,NA,13343.0,294355.0,9241.0,"[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...","[[(cell, 7869), (protein, 3832), (membran, 334...",92.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",924.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",1848.0,"[[(cell, 7869), (protein, 3832), (membran, 334...",2310.0,"[[(cell, 7869), (protein, 3832), (membran, 334..."
178,2020,NA,16042.0,353675.0,9992.0,"[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...","[[(cell, 9252), (protein, 4577), (membran, 404...",100.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",999.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",1998.0,"[[(cell, 9252), (protein, 4577), (membran, 404...",2498.0,"[[(cell, 9252), (protein, 4577), (membran, 404..."
179,2021,NA,16459.0,362228.0,10134.0,"[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...","[[(cell, 9165), (protein, 4849), (membran, 400...",101.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",1013.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",2027.0,"[[(cell, 9165), (protein, 4849), (membran, 400...",2534.0,"[[(cell, 9165), (protein, 4849), (membran, 400..."
180,2022,NA,7218.0,159473.0,6572.0,"[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...","[[(cell, 3813), (protein, 2282), (membran, 161...",66.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",657.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",1314.0,"[[(cell, 3813), (protein, 2282), (membran, 161...",1643.0,"[[(cell, 3813), (protein, 2282), (membran, 161..."
